In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool, tool
from langchain.agents import initialize_agent

from pydantic import BaseModel, Field
from math import pi, sqrt, cos, sin

## Setting the LLM

In [ ]:
chat_llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0,
    openai_api_key = open("openai_api.txt", "r").read()
)

## Setting the Memory Object

In [ ]:
conversational_memory = ConversationBufferWindowMemory(
    memory_key = "chat_history",
    k = 5,
    return_messages = True # return chat history when agent perform a task
)

## Simple Calculator Tool


We will start with a simple custom tool. The tool is a simple calculator that calculates a circle's circumference based on the circle's radius.

* `BaseTool` - the required template for a LangChain Tool.

* `name`, `description` - the required parameters so LangChain can recognise an object as a valid Tool.
    * In our `description`, we did not define when `not` to use the tool. That is because the LLM seemed capable of identifying when this tool is needed. Adding “when not to use it” to the description can help if a tool is overused.

* `_run`, `_arun` - when a Tool is being used, "_run" is being called by default.



In [ ]:
class CircumferenceTool(BaseTool):
    name = "Circumference Calculator"
    description = "Use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius):
        return 2 * pi * float(radius)

    def _arun(self, radius):
        raise NotImplementedError("[ERROR] This tool does not support async...")

In [ ]:
## Initializing Agent

tools = [CircumferenceTool()]

agent = initialize_agent(
    agent = "chat-conversational-react-description",
    tools = tools,
    llm = chat_llm,
    verbose = True,
    max_iterations = 3,
    early_stopping_method = "generate",
    memory = conversational_memory
)

In [ ]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

In [ ]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Changing the Prompt to Tell the LLM that it doesn't know how to perform math, so it needs to search in its tools.

In [ ]:
sys_message = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, the Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to its trusty tools and absolutely does NOT try to answer math questions by itself.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [ ]:
## Creating new Prompt

new_prompt = agent.agent.create_prompt(
    system_message = sys_message,
    tools = tools
)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")

* We can see that the agent now uses the `Circumference calculator` tool and consequently gets the correct answer.

## Using the `tool` Decorator

To make it easier to define custom tools, a `@tool` decorator is provided. This decorator can be used to quickly create a Tool from a simple function. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description.

In [ ]:
@tool
def search_api(query: str) -> str:
    """Searches the API for the query"""
    return f"Results for query {query}"

search_api

In [ ]:
@tool("search", return_direct=True)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

search_api

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search", return_direct=True, args_schema=SearchInput)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

search_api

## Tools with Mupltiple Parameters

We are going to build a `Hypotenuse Calculator`.

We want multiple inputs here because we calculate the triangle hypotenuse with different values.

In [ ]:
class PythagorasTool(BaseTool):
    name = "Hypotenuse Calculator"
    description = """Use this tool when you need to calculate the length of a hypotenuse given one or two sides of a triangle and/or an angle (in degrees).
        To use the tool, you must provide at least two of the following parameters ['adjacent_side', 'opposite_side', 'angle'].
    """

    def _run(self, adjacent_side=None, opposite_side=None, angle=None):
        if (adjacent_side is not None) and (opposite_side is not None):
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif (adjacent_side is not None) and (angle is not None):
            return adjacent_side / cos(float(angle))
        elif (opposite_side is not None) and (angle is not None):
            return opposite_side / sin(float(angle))
        else:
            return "[ERROR] Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."

    def _arun(self, query: str):
        raise NotImplementedError("[ERROR] This tool does not support async.")

In [ ]:
## Adding this new Tool

tools.append(PythagorasTool())

As before, we must update the agent's prompt. We don't need to modify the system message as we did before, but we do need to update the available tools described in the prompt.

In [ ]:
new_prompt = agent.agent.create_prompt(
    system_message = sys_message,
    tools = tools
)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
## Adding new Tools to Agent

agent.tools = tools

In [ ]:
agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")

In [ ]:
agent("If I have a triangle with the opposite side of length 51cm and an angle of 20 deg, what is the length of the hypotenuse?")